In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [3]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))

In [4]:
#Create a pool of data
train_pool_b = Pool(X_train_new_b, y_train_new_b)

test_pool_b = Pool(X_test_new_b) 

In [5]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_b = CatBoostRegressor(**params, verbose=300)
    catboost_model_b.fit(train_pool_b)
    pred_b = pd.DataFrame(catboost_model_b.predict(test_pool_b))
    MAE_b = mean_absolute_error(y_test_b, pred_b)
    return MAE_b

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-11 09:29:07,100] A new study created in memory with name: no-name-355a4b7c-78e3-4ce1-b5dd-63e5061a5a50


0:	learn: 79.2382445	total: 74ms	remaining: 2m 11s
300:	learn: 44.3742865	total: 8.48s	remaining: 41.7s
600:	learn: 30.7834734	total: 17.7s	remaining: 34.8s
900:	learn: 26.2701912	total: 26.9s	remaining: 26.3s
1200:	learn: 24.5600678	total: 36.4s	remaining: 17.6s
1500:	learn: 23.8302484	total: 45.9s	remaining: 8.55s


[I 2023-11-11 09:30:02,000] Trial 0 finished with value: 64.68143998688977 and parameters: {'iterations': 1781, 'learning_rate': 0.00316305288984533, 'depth': 7, 'min_data_in_leaf': 87, 'l2_leaf_reg': 8, 'bagging_temperature': 0.6365952101833992, 'random_strength': 0.41199889300071607, 'border_count': 661, 'rsm': 0.7822835898455944}. Best is trial 0 with value: 64.68143998688977.


1780:	learn: 23.3691317	total: 54.6s	remaining: 0us
0:	learn: 76.4029604	total: 158ms	remaining: 2m 39s
300:	learn: 5.1958071	total: 44.3s	remaining: 1m 44s
600:	learn: 1.6987951	total: 1m 27s	remaining: 60s
900:	learn: 0.7310776	total: 2m 15s	remaining: 16.7s
300:	learn: 15.3074351	total: 25.9s	remaining: 58.2s
600:	learn: 9.6984728	total: 52s	remaining: 32.6s
900:	learn: 6.8505228	total: 1m 18s	remaining: 6.71s
977:	learn: 6.3289259	total: 1m 25s	remaining: 0us


[I 2023-11-11 09:34:01,111] Trial 2 finished with value: 66.83657703909519 and parameters: {'iterations': 978, 'learning_rate': 0.03163956523997024, 'depth': 12, 'min_data_in_leaf': 89, 'l2_leaf_reg': 9, 'bagging_temperature': 0.8691211860181103, 'random_strength': 0.9455785452956014, 'border_count': 436, 'rsm': 0.322807769113986}. Best is trial 0 with value: 64.68143998688977.


...

1884:	learn: 18.6768882	total: 43.4s	remaining: 0us
0:	learn: 77.8592341	total: 19.5ms	remaining: 39.3s
300:	learn: 22.2625310	total: 6.65s	remaining: 38s
600:	learn: 19.7778093	total: 13.6s	remaining: 32.2s
900:	learn: 18.1594649	total: 20.2s	remaining: 25.1s
1200:	learn: 16.8466934	total: 27s	remaining: 18.4s
1500:	learn: 15.8153380	total: 33.6s	remaining: 11.6s
1800:	learn: 14.8742958	total: 40.4s	remaining: 4.91s


[I 2023-11-11 11:20:27,345] Trial 78 finished with value: 64.08782583525006 and parameters: {'iterations': 2020, 'learning_rate': 0.03309591884727989, 'depth': 7, 'min_data_in_leaf': 37, 'l2_leaf_reg': 3, 'bagging_temperature': 0.6484007242484774, 'random_strength': 0.383546209025953, 'border_count': 22, 'rsm': 0.9730372562830846}. Best is trial 28 with value: 61.79832585348951.


2019:	learn: 14.2837355	total: 45.3s	remaining: 0us
0:	learn: 78.5872706	total: 17.2ms	remaining: 30.7s
300:	learn: 24.4975561	total: 6.25s	remaining: 31s
600:	learn: 22.9708640	total: 12.5s	remaining: 24.8s
900:	learn: 21.8590930	total: 19s	remaining: 18.8s
1200:	learn: 20.9704691	total: 25.4s	remaining: 12.5s
1500:	learn: 20.0599171	total: 31.9s	remaining: 6.2s


[I 2023-11-11 11:21:06,059] Trial 79 finished with value: 62.10989272213895 and parameters: {'iterations': 1793, 'learning_rate': 0.017821419908582544, 'depth': 6, 'min_data_in_leaf': 54, 'l2_leaf_reg': 3, 'bagging_temperature': 0.6265480675064097, 'random_strength': 0.42192324034351925, 'border_count': 112, 'rsm': 0.8139631990232552}. Best is trial 28 with value: 61.79832585348951.


1792:	learn: 19.4599298	total: 38.4s	remaining: 0us


In [9]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_b.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1480, 'learning_rate': 0.004154859937303772, 'depth': 1, 'colsample_bylevel': 0.8980589862889544, 'min_data_in_leaf': 11, 'l2_leaf_reg': 7, 'has-time': True, 'bagging_temperature': 0.8588447919947089, 'random_strength': 0.5000970078140154, 'border_count': 358, 'rsm': 0.31318686704086646, 'nan_mode': 'min'}
82.32735192987839
